# Simulating sets of data using RocketPy

### Setting up the simulation

In [ ]:
from rocketpy import Environment, Flight, Rocket, Fluid, CylindricalTank, MassFlowRateBasedTank, HybridMotor
%config InlineBackend.figure_formats = ['svg']
%matplotlib inline


############################################################################
#Engine Settings 

# Define the fluids
oxidizer_liq = Fluid(name="N2O_l", density=1220)
oxidizer_gas = Fluid(name="N2O_g", density=1.9277)

# Define tank geometry
tank_shape = CylindricalTank(115 / 2000, 0.705)

# Define tank
oxidizer_tank = MassFlowRateBasedTank(
    name="oxidizer tank",
    geometry=tank_shape,
    flux_time=5.2,
    initial_liquid_mass=4.11,
    initial_gas_mass=0,
    liquid_mass_flow_rate_in=0,
    liquid_mass_flow_rate_out=(4.11 - 0.5) / 5.2,
    gas_mass_flow_rate_in=0,
    gas_mass_flow_rate_out=0,
    liquid=oxidizer_liq,
    gas=oxidizer_gas,
)

example_hybrid = HybridMotor(
    thrust_source=lambda t: 2000 - (2000 - 1400) / 5.2 * t,
    dry_mass=2,
    dry_inertia=(0.125, 0.125, 0.002),
    nozzle_radius=63.36 / 2000,
    grain_number=4,
    grain_separation=0,
    grain_outer_radius=0.0575,
    grain_initial_inner_radius=0.025,
    grain_initial_height=0.1375,
    grain_density=900,
    grains_center_of_mass_position=0.384,
    center_of_dry_mass_position=0.284,
    nozzle_position=0,
    burn_time=5.2,
    throat_radius=26 / 2000,
)

example_hybrid.add_tank(
  tank = oxidizer_tank, position = 1.0615
)


############################################################################
#Rocket Configuration 

# lines with ## at the end have been matched with openrocket design metrics
calisto = Rocket(
    radius=196 / 2000, ##
    mass=28.55, ##
    inertia=(6.321, 6.321, 0.034),
    power_off_drag="powerOffDragCurve.csv",
    power_on_drag="powerOnDragCurve.csv",
    center_of_mass_without_motor=3.39, ##
    coordinate_system_orientation="nose_to_tail", ##
)

rail_buttons = calisto.set_rail_buttons(
    upper_button_position=3.98, ##
    lower_button_position=5.25, ##
    angular_position=135, ##
)
nose_cone = calisto.add_nose(length=0.902, kind="vonKarman", position=0.0) ##
fin_set = calisto.add_trapezoidal_fins(
    n=4,##
    root_chord=0.495,##
    tip_chord=0.152,##
    span=0.229,##
    position=4.78,##
    cant_angle=0,##
    airfoil=("NACA0012-radians.txt", "radians"),
)
tail = calisto.add_tail(
    top_radius=0.098, bottom_radius=0.0705, length=0.17, position=5.28 ##
)
main = calisto.add_parachute(
    "Main",
    cd_s=42,##
    trigger=800,
    sampling_rate=105,
    lag=1.5,
    noise=(0, 8.3, 0.5),
)
drogue = calisto.add_parachute(
    "Drogue",
    cd_s=1.8, ##
    trigger="apogee",
    sampling_rate=105,
    lag=1.5,
    noise=(0, 8.3, 0.5),
)

calisto.add_motor(example_hybrid, position=1.43)##

In [11]:
import datetime
############################################################################
#Environment Settings 

env = Environment(latitude=32.990254, longitude=-106.974998, elevation=1400)
tomorrow = datetime.date.today() + datetime.timedelta(days=1)
env.set_date(
    (tomorrow.year, tomorrow.month, tomorrow.day, 12)
)  # Hour given in UTC time
env.set_atmospheric_model(type="Forecast", file="GFS")

In [ ]:
from rocketpy.simulation.flight_data_exporter import FlightDataExporter

test_flight = Flight(
    rocket=calisto, environment=env, rail_length=5.2, inclination=85, heading=0
)

# Create an exporter object
exporter = FlightDataExporter(test_flight)

exporter.export_data(
    "calisto_flight_data.csv",
    "acceleration",
    "vz",
    "z",
)

print("Flight data exported to flight_data.csv")

Flight data exported to flight_data.csv
